In [1]:
import pandas as pd
from neo4j import GraphDatabase, basic_auth
from py2neo import Graph
from py2neo.data import Node, Relationship

In [2]:
#Cloud
#graph = Graph(scheme="bolt", host="54.90.165.10", port="7687", user="neo4j", password="clicks-haul-qualifier")

In [3]:
#localhost
graph = Graph(scheme="bolt", host="localhost", port="7787", user="neo4j", password="neo4j1")

Recupero utenti collegati a tweets contenente le parole chiave elencate

In [4]:
query = '''
WITH [ "gme", "gamestonk", "squeeze", "robinhood", "wallstreetbets", "wsb"] AS terms
MATCH (t:TwUser)-[x:TWEETED]->(c:Tweet)
MATCH (t:TwUser)<-[y:MENTIONED]-(s:Tweet)
WHERE any(word IN terms WHERE toLower(c.text) CONTAINS word)
OR any(word IN terms WHERE toLower(s.text) CONTAINS word)
RETURN DISTINCT t.degree, t.username
'''
result = graph.run(query)
twitterUsers = pd.DataFrame(result, columns={'degree', 'username'})
twitterUsers.sort_values(by=['degree'], ascending=False)

,degree,username
30798,26622.0,RobinhoodApp
51,22465.0,GameStop
30623,12528.0,elonmusk
30787,7048.0,chamath
2371,6994.0,WSBChairman
...,...,...
29421,0.0,TedKnutsonDC
37089,0.0,tezawaly
37095,0.0,EAlcaz
37098,0.0,DappUniversity


In [5]:
twitterUsers.to_csv('../data/from_db/twitter_users.csv', sep=',', index=False)

In [6]:
query = '''
WITH [ "gme", "gamestonk", "squeeze", "robinhood", "wallstreetbets", "wsb"] AS terms
match (t:Tweet)
WHERE any(word IN terms WHERE toLower(t.text) CONTAINS word)
return t.date,
t.tweetId,
t.userId,
t.display_name,
t.text,
t.reply_count,
t.like_count,
t.retweet_count,
t.quote_count,
t.retweted_tweet,
t.username,
t.degree
'''
result = graph.run(query)
tweets = pd.DataFrame(result)
tweets.rename(columns={0:'date', 1:'tweetId', 2:'userId', 3:'display_name', 
                        4:'text', 5:'reply_count', 6:'like_count', 7:'retweet_count',
                        8:'quote_count', 9:'retweted_tweet', 10:'username',
                        11:'degree'}, inplace=True)

In [7]:
tweets.to_csv('../data/from_db/tweets.csv', sep=',', index=False)

In [8]:
tweets.sort_values(by=['like_count'], ascending=False)

,date,tweetId,userId,display_name,text,reply_count,like_count,retweet_count,quote_count,retweted_tweet,username,degree
175429,2021-01-26 21:37:36+00:00,1354181719432290306,1163605918615035904,The StockCast,If $GME Somehow hits $500... I will name my ne...,82,999,48,14,None,TheStockCast,0.0
568770,2021-01-29 15:50:53+00:00,1355181632676880385,204089551,Tom,THE ROCKET IS FUCKING FUELED! #1337 TO THE MOO...,80,999,95,4,None,Syndicate,0.0
7841,2021-01-26 21:53:29+00:00,1354185719967854594,19040580,Rod Breslau,lmao GameStop hits 230 after hours and is now ...,79,9979,597,133,None,Slasher,0.0
47335,2021-01-28 16:09:40+00:00,1354823970915196933,782641035268595712,Spawn Wave,So how long can Robinhood realistically keep G...,44,997,41,2,None,SpawnWaveMedia,0.0
303151,2021-01-28 01:23:50+00:00,1354601040629428226,1226189022,LoadingMeerkat,HOLD THE LINE 🚀 There’s lower trade volume aft...,30,997,259,20,None,LoadingMeerkat,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
406620,2021-01-28 16:14:46+00:00,1354825253143384064,515508200,appe,i want to buy gme at 150 but i cant buy😡,1,0,0,0,None,APproctor,0.0
305591,2021-01-28 01:47:36+00:00,1354607022810025985,1280632735627952130,gann0x Multiverse,@UnfilteredAfc $amc $gme,0,0,0,0,None,gann0x,1.0
27934,2021-01-28 01:45:18+00:00,1354606446873354244,1087312824395800577,Ian Culley,For those who might lack the temperament to ha...,,,,,None,IanCulley,0.0
495021,2021-01-28 22:59:13+00:00,1354927035722960897,868802550,James Young,Other side of the GME takedown story,,,,,None,YoungEcon,0.0


In [9]:
tweets[tweets['username'] == 'elonmusk']

,date,tweetId,userId,display_name,text,reply_count,like_count,retweet_count,quote_count,retweted_tweet,username,degree
171719,2021-01-26 21:08:02+00:00,1354174279894642703,44196397,Elon Musk,Gamestonk!! https://t.co/RZtkDzAewJ,11685,240616,34223,9345,None,elonmusk,0.0


<br/>
Recupero twitter users per visualizzazione grafica di Neo4J

In [6]:
query = '''
WITH [ "gme", "gamestonk", "squeeze", "robinhood", "wallstreetbets", "wsb"] AS terms
match (t:TwUser)-[x:TWEETED]->(c:Tweet)
match (tu:TwUser)<-[z:HAS_MENTIONED]-(t)<-[y:MENTIONED]-(s:Tweet)
WHERE (any(word IN terms WHERE toLower(c.text) CONTAINS word)
OR any(word IN terms WHERE toLower(s.text) CONTAINS word))
AND t.degree > 200
return distinct t, tu limit 450
'''
result = graph.run(query)

In [9]:
conn.close()